In [1]:
%pip install python-binance

from binance.client import Client as bnb_client
from datetime import datetime
import pandas as pd 
import pickle
import numpy as np 

#client = bnb_client()
###  if you're in the US, use: 
client = bnb_client(tld='US')#" here instead

def get_binance_px(symbol,freq='1d',start_ts = '2025-03-03',end_ts='2025-22-04'):
    data = client.get_historical_klines(symbol,freq,start_ts,end_ts)
    columns = ['open_time','open','high','low','close','volume','close_time','quote_volume',
    'num_trades','taker_base_volume','taker_quote_volume','ignore']

    data = pd.DataFrame(data,columns = columns)
    
    # Convert from POSIX timestamp (number of millisecond since jan 1, 1970)
    data['open_time'] = data['open_time'].map(lambda x: datetime.utcfromtimestamp(x/1000))
    data['close_time'] = data['close_time'].map(lambda x: datetime.utcfromtimestamp(x/1000))
    return data 

univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']

px = {}
for x in univ:
    data = get_binance_px(x)
    px[x] = data.set_index('open_time')['close']

px = pd.DataFrame(px).astype(float)
px = px.reindex(pd.date_range(px.index[0],px.index[-1],freq='1d'))
ret = px.pct_change()


Note: you may need to restart the kernel to use updated packages.


C:\Users\rohan\AppData\Local\Temp\ipykernel_14976\1087420948.py:34: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = px.pct_change()


In [ ]:
with open('../data/cryptoData.pkl','wb') as file:
    pickle.dump(px,file)


In [3]:
freq = '1h'
px = {}
for x in univ:
    data = get_binance_px(x,freq)
    px[x] = data.set_index('open_time')['close']
px = pd.DataFrame(px).astype(float)
px = px.reindex(pd.date_range(px.index[0],px.index[-1],freq=freq))
ret = px.pct_change()


C:\Users\rohan\AppData\Local\Temp\ipykernel_14976\4076592346.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = px.pct_change()


In [ ]:
today=datetime.now()
with open('../data/cryptoDataHourly'+today.strftime("%Y_%m_%d_%H_%M")+'.pkl','wb') as file:
    pickle.dump(px,file)


In [5]:
ret.tail()

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2025-04-21 20:00:00,-0.001303,0.000635,0.000000,-0.003442,-0.002105,0.000522,NaN
2025-04-21 21:00:00,0.000133,0.000857,-0.001124,-0.000084,-0.003116,-0.004691,NaN
2025-04-21 22:00:00,-0.002814,0.000679,-0.002250,0.001744,-0.001827,-0.006023,NaN
2025-04-21 23:00:00,0.004719,0.002016,0.005638,0.000469,0.004770,-0.004742,NaN
2025-04-22 00:00:00,0.002213,-0.008972,-0.002883,-0.001355,-0.004076,-0.006617,NaN


In [6]:
len(ret)

1201